In [1]:
#importing the Libraies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# Reading the Dataset
dataset = pd.read_csv('/content/Social_Network_Ads.csv')

In [3]:
dataset.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [4]:
dataset.tail()

,User ID,Gender,Age,EstimatedSalary,Purchased
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0
399,15594041,Female,49,36000,1


In [5]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
X=dataset[['Age', 'EstimatedSalary','Gender_Male']]
y=dataset['Purchased']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier()

In [11]:
# 6. Define Parameter Distributions for Randomized Search
param_dist = {
    'n_neighbors': np.arange(1, 31),           # Number of neighbors
    'weights': ['uniform', 'distance'],        # Weight function
    'metric': ['euclidean', 'manhattan', 'minkowski']  # Distance metrics
}

# 7. RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=knn,
    param_distributions=param_dist,
    n_iter=20,               # Number of random combinations to try
    scoring='accuracy',
    cv=3,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

# Fit the model
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=3, estimator=KNeighborsClassifier(), n_iter=20, n_jobs=-1,
                   param_distributions={'metric': ['euclidean', 'manhattan',
                                                   'minkowski'],
                                        'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]),
                                        'weights': ['uniform', 'distance']},
                   random_state=42, scoring='accuracy', verbose=1)

In [12]:
# Cross-validated predictions on the test set
from sklearn.model_selection import cross_val_predict
y_pred = random_search.best_estimator_.predict(X_test)

In [13]:
# Evaluation
from sklearn.metrics import confusion_matrix, classification_report, f1_score
cm = confusion_matrix(y_test, y_pred)
clf_report = classification_report(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average='weighted')

In [14]:
print("Best Parameters:", random_search.best_params_)
print("The f1_macro:", f1_macro)
print("The confusion Matrix:\n", cm)
print("The report:\n", clf_report)

Best Parameters: {'weights': 'uniform', 'n_neighbors': np.int64(19), 'metric': 'manhattan'}
The f1_macro: 0.7448309178743961
The confusion Matrix:
 [[48  4]
 [15 13]]
The report:
               precision    recall  f1-score   support

           0       0.76      0.92      0.83        52
           1       0.76      0.46      0.58        28

    accuracy                           0.76        80
   macro avg       0.76      0.69      0.71        80
weighted avg       0.76      0.76      0.74        80



In [16]:
# Save Best Model
import pickle
filename = "KNN_random_search_best_model.sav"
pickle.dump(random_search.best_estimator_, open(filename, 'wb'))

In [17]:
# CV Results table
results_df = pd.DataFrame(random_search.cv_results_)
display(results_df.head())

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_weights,param_n_neighbors,param_metric,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.004983,0.001198,0.005695,0.000534,distance,10,euclidean,"{'weights': 'distance', 'n_neighbors': 10, 'me...",0.775701,0.803738,0.707547,0.762329,0.040392,12
1,0.003204,0.000032,0.005026,0.000046,uniform,22,euclidean,"{'weights': 'uniform', 'n_neighbors': 22, 'met...",0.822430,0.822430,0.754717,0.799859,0.031920,8
2,0.003369,0.000185,0.004683,0.000782,distance,17,minkowski,"{'weights': 'distance', 'n_neighbors': 17, 'me...",0.766355,0.794393,0.707547,0.756098,0.036189,15
3,0.003647,0.000675,0.005835,0.000380,uniform,10,manhattan,"{'weights': 'uniform', 'n_neighbors': 10, 'met...",0.831776,0.813084,0.764151,0.803004,0.028513,6
4,0.004170,0.000556,0.004491,0.000555,distance,13,minkowski,"{'weights': 'distance', 'n_neighbors': 13, 'me...",0.766355,0.794393,0.707547,0.756098,0.036189,15
